<a href="https://colab.research.google.com/github/luasm17/LLM_as_a_judge/blob/main/prueba_2_prometheus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar dependencias + Imports
!pip install -q -U "transformers>=4.51.0" accelerate safetensors

import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 28.7 MB/s eta 0:00:00


In [2]:
# Token HF
HF_TOKEN = os.environ.get("HF_TOKEN", None)

In [3]:
# Cargar modelo e tokenizer (M-Prometheus-7B)
# LLM-as-a-Judge binario para concordancia de número en galego

MODEL_ID = "Unbabel/M-Prometheus-7B"

print("🔄 Cargando tokenizer...")
if HF_TOKEN:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
else:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

print("🔄 Cargando modelo (pode tardar uns minutos)...")
if HF_TOKEN:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        token=HF_TOKEN,
        device_map="auto",
        torch_dtype="auto"
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto",
        torch_dtype="auto"
    )

model.eval()
print("✅ Modelo cargado correctamente")

🔄 Cargando tokenizer...


config.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

✅ Modelo cargado correctamente


In [4]:
# Función LLM-as-a-Judge (concordancia de número)
def prometheus_judge_concordancia(oracion: str) -> str:
    prompt = f"""
You are an LLM-as-a-judge evaluating the output of a Galician grammatical error correction (GEC) model.

Your task: decide whether the given sentence contains an error of NUMBER AGREEMENT
(singular/plural agreement between determiner, noun, adjective, or verb).

You MUST evaluate ONLY number agreement. Do NOT evaluate any other error types.
You MUST NOT, under any circumstances, correct or rewrite the sentence. Only judge it.

Rubric (binary):
- [RESULT] 1 -> The correction is NOT adequate (there is still a number agreement error)
- [RESULT] 0 -> The correction is adequate (there is NO number agreement error)

Output format (STRICT, no extra text):
Feedback: "<brief and precise justification in Galician focused ONLY on number agreement>"
[RESULT] <0 or 1>

Now evaluate the following GEC model output:

<OUTPUT_MODELO>
{oracion}
</OUTPUT_MODELO>
"""

    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.0,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    resposta = tokenizer.decode(
        output[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )

    return resposta.strip()

In [5]:
# Probas cos exemplos
exemplos = [
    "As decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas.",
    "O grupos de estudantes que participou no proxecto presentou os resultados finais onte pola tarde.",
    "As propostas que chegaron desde os concellos máis pequenos foi analizadas polo equipo técnico.",
    "A maioría das persoas entrevistadas manifestou a súa opinión durante a sesión pública.",
    "Os relatorios anuais sobre o impacto ambiental foi revisados exhaustivamente pola comisión de avaliación.",
    "A valiosa colección de poemas inéditos do autor galego publicouse recentemente baixo un prestixioso selo independente.",
    "O paquete de medidas económicas recentemente aprobado é de aplicación inmediata en todos os sectores da economía.",
    "As adversas condicións climáticas dos últimos días obrigou a suspender completamente varias actividades programadas ao aire libre.",
    "A maioría dos participantes no curso de formación mostrou un grande interese en continuar con sesións prácticas adicionais.",
    "Os sabios consellos que me deches sobre a xestión do tempo foi moi útiles para poder tomar unha decisión acertada.",
]

for i, frase in enumerate(exemplos, 1):
    print(f"\n===== EXEMPLO {i} =====")
    print(prometheus_judge_concordancia(frase))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



===== EXEMPLO 1 =====
Feedback: "A frase está correcta en termos de número concordante. O verbo 'foron' está adequadamente concordado coa forma plural 'decisións', e o artigo 'aos' tamén está correcto para o plural 'responsables'. A concordancia entre os elementos nominais é precisa."
[RESULT] 0

===== EXEMPLO 2 =====
Feedback: "A frase contén un erro de número na concordancia do verbo 'presentou'. O verbo debe estar en plural para concordar coa frase 'grupos de estudantes', polo que a oración correcta sería 'presentaron'." 
[RESULT] 1

===== EXEMPLO 3 =====
Feedback: "A frase contén un erro de número en 'foi analizadas', onde o verbo está em plural, pero o sujeito 'as propostas' é singular. O verbo debe estar em singular para concordar coa concordancia do sujeito."
[RESULT] 1

===== EXEMPLO 4 =====
Feedback: "O número de 'persoas' está en concordancia coa preposición 'das', pero o verbo 'manifestou' está en singular, que non concorda coa pluralidade do sujeito 'as persoas'. A frase t